In [ ]:
# Mount Google Drive to access the required folders
from google.colab import drive
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from tqdm import tqdm

# Mount the Google Drive
drive.mount('/content/drive', force_remount=True)

base_folder = '/content/drive/Shared drives/Enverus US'

output_folder = os.path.join(base_folder, 'Processed Data')
raw_wells_production_folder = os.path.join(base_folder, 'Well Monthly Production - Anadarko')
raw_wells_headers_folder = os.path.join(base_folder, 'Well Headers - Anadarko')
wells_production_folder = os.path.join(output_folder, 'Well Monthly Production with useful columns - Anadarko')
wells_headers_folder = os.path.join(output_folder, 'Well Headers with useful columns - Anadarko')

# Create the output directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

print(f"Base folder: {base_folder}")
print(f"Wells production folder: {wells_production_folder}")
print(f"Wells headers folder: {wells_headers_folder}")
print(f"Output folder: {output_folder}")

# Step 1: Process raw production files

# Create a folder for processed files
os.makedirs(wells_production_folder, exist_ok=True)

# Filter files that match the specific names
production_csv_files = [f for f in os.listdir(raw_wells_production_folder)]

# Process each file individually
for file in tqdm(production_csv_files, desc="Processing CSV files"):
    # Read only the necessary columns from the original file
    file_path = os.path.join(raw_wells_production_folder, file)
    print(f'Processing {file_path}')
    processed_df = pd.read_csv(
        file_path,
        usecols=['API/UWI', 'Monthly Oil', 'Monthly Gas', 'Monthly Production Date']
    )
    processed_df = processed_df[~pd.isna(processed_df['API/UWI'])]

    processed_df['API/UWI'] = processed_df['API/UWI'].astype(str)
    # remove API = 0 because many occurrences
    processed_df = processed_df[processed_df['API/UWI'] > '0']

    # Save the processed file to the new folder
    output_path = os.path.join(wells_production_folder, file)
    processed_df.to_csv(output_path, index=False)

print(f"Processed files saved to: {wells_production_folder}")

# Step 2: Process raw well headers files

# Create a folder for processed files
os.makedirs(wells_headers_folder, exist_ok=True)

headers_csv_files = [f for f in os.listdir(raw_wells_headers_folder) if f.lower().endswith('.csv')]

# Process each file individually
for file in tqdm(headers_csv_files, desc="Processing CSV files"):
    # Read only the necessary columns from the original file
    file_path = os.path.join(raw_wells_headers_folder, file)
    processed_df = pd.read_csv(
        file_path,
        usecols=['API14', 'Surface Hole Latitude (WGS84)', 'Surface Hole Longitude (WGS84)']
    )

    processed_df = processed_df[~pd.isna(processed_df['API14'])]
    processed_df['API14'] = processed_df['API14'].astype(str)
    # remove API = 0
    processed_df = processed_df[processed_df['API14'] != '0']

    # Save the processed file to the new folder
    output_path = os.path.join(wells_headers_folder, file)
    processed_df.to_csv(output_path, index=False)

print(f"Processed files saved to: {wells_headers_folder}")

# Step 3: Merge well header files with basins info

def process_well_headers(headers_folder, basin_gdf, output_folder):
    output_file = os.path.join(output_folder, 'wellheaderswithbasins.csv')
    print(output_file)
    print("Processing well header files...")
    headers_files = [f for f in os.listdir(headers_folder) if f.lower().endswith('.csv')]

    # Concatenate well header files
    headers_df = pd.concat(
        [pd.read_csv(os.path.join(headers_folder, file)) for file in tqdm(headers_files, desc="Reading headers")],
        ignore_index=True
    )

    # Convert to GeoDataFrame
    headers_gdf = gpd.GeoDataFrame(
        headers_df,
        geometry=[Point(xy) for xy in zip(headers_df['Surface Hole Longitude (WGS84)'], headers_df['Surface Hole Latitude (WGS84)'])],
        crs='EPSG:4326'
    )
    # First convert to the same crs (!)
    # Spatial join with basins
    headers_gdf = gpd.sjoin(headers_gdf.to_crs(26914), basin_gdf.to_crs(26914), how='inner', predicate='within')

    # Save as a Pandas DataFrame with BASIN_NAME column
    headers_gdf[['API14', 'Surface Hole Latitude (WGS84)', 'Surface Hole Longitude (WGS84)', 'BASIN_NAME']].to_csv(output_file, index=False)
    print(f"Processed well headers saved to: {output_file}")
    return output_file

# Step 4: Merge production files with well headers (that have basin infos) to create initial output files

def process_production_files(production_folder, headers_file, output_folder):
    print("Reading Well Headers with Basins file...")
    headers_df = pd.read_csv(headers_file)

    print("Looking at list of files...")

    production_files = [f for f in os.listdir(production_folder) if f.lower().endswith('.csv')]
    print("Processing production files...")
    for file in tqdm(production_files, desc="Processing production files"):
        production_df = pd.read_csv(os.path.join(production_folder, file))
        production_df['API/UWI'] = production_df['API/UWI'].astype(str)
        headers_df['API14'] = headers_df['API14'].astype(str)
        print(f"Read file {file}. Merging with headers...")

        # Merge with headers
        merged_df = pd.merge(
            production_df,
            headers_df,
            left_on='API/UWI',
            right_on='API14',
            how='left'
        )
        print("Saving...")

        # Save each merged file
        output_file = os.path.join(output_folder, f"processed_{file}")
        merged_df.to_csv(output_file, index=False)

        print(f"Processed production file saved to: {output_file}")

# Step 5: Split each initial output file into subfiles by BASIN_NAME

def split_by_basin(input_folder, output_folder):
    input_files = [f for f in os.listdir(input_folder) if f.lower().endswith('.csv')]
    print('Looping through files...')
    for file in tqdm(input_files, desc="Splitting by BASIN_NAME"):
        if 'KS' in file:
          df = pd.read_csv(os.path.join(input_folder, file))
          print(f'Uploaded {file}')
          # Split by BASIN_NAME
          for basin_name, group in df.groupby('BASIN_NAME'):
              print(basin_name)
              basin_filename = f"{basin_name.replace(' ', '_')}_{file}"
              basin_output_path = os.path.join(output_folder, basin_filename)
              group.to_csv(basin_output_path, index=False)
              print(f"Saved basin file: {basin_output_path}")

# Step 6: Concatenate subfiles by BASIN_NAME

def concat_by_basin(input_folder, final_output_folder, basins_gdf, processed_files=None):
    """
    Concatenate files by BASIN_NAME and track processed files.

    Args:
        input_folder (str): Folder containing input CSV files.
        final_output_folder (str): Folder to save final concatenated CSV files.
        basins_gdf (GeoDataFrame): GeoDataFrame with BASIN_NAME column.
        processed_files (list): List of already processed files (optional).
    """
    print("Concatenating files by BASIN_NAME...")

    # Initialize the processed files list if not provided
    if processed_files is None:
        processed_files = []

    # Loop through all unique basin names
    for basin_name in tqdm(basins_gdf.BASIN_NAME.unique(), desc="Processing basins"):
        print(f'Looking at {basin_name}...')

        # Create a formatted version of the basin name to match filenames
        formatted_basin_name = basin_name.replace(' ', '_')

        # Find all files containing the basin name in their filename
        matching_files = [
            f for f in os.listdir(input_folder)
            if f.lower().endswith('.csv') and f.split('_processed')[0] == formatted_basin_name
        ]

        # Filter out files that are already processed
        matching_files = [f for f in matching_files if f not in processed_files]

        # Skip if no matching files found
        if not matching_files:
            print(f"No unprocessed files found for basin: {basin_name}")
            continue

        print(f"{len(matching_files)} unprocessed files found for {basin_name}, concatenating them...")

        # Concatenate all matching files
        dfs = []
        for file in matching_files:
            file_path = os.path.join(input_folder, file)
            dfs.append(pd.read_csv(file_path))
            # Add the file to the processed files list
            processed_files.append(file)

        final_df = pd.concat(dfs, ignore_index=True)

        # Replace formatted names with the original BASIN_NAME
        final_df['BASIN_NAME'] = basin_name
        print('Saving the file...')

        # Save the final DataFrame with the original BASIN_NAME in the filename
        final_file = os.path.join(final_output_folder, f"{basin_name}_final.csv")
        final_df.to_csv(final_file, index=False)
        print(f"Final concatenated file saved for basin {basin_name}: {final_file}")

        # Clear memory for this basin
        del dfs, final_df

    print("All basins processed.")
    return processed_files


# Paths

headers_output_folder = os.path.join(output_folder, 'Well Headers with Basins')
merged_output_folder = os.path.join(output_folder, 'Well Monthly Production with Headers and Basins')
split_output_folder = os.path.join(output_folder, 'Well Monthly Production with Headers and Basins split by Basin')
final_output_folder = os.path.join(output_folder, 'Well Monthly Production with Headers and Basins concatenated by Basin')

# Create the output directory if it doesn't exist
os.makedirs(headers_output_folder, exist_ok=True)
os.makedirs(merged_output_folder, exist_ok=True)
os.makedirs(split_output_folder, exist_ok=True)
os.makedirs(final_output_folder, exist_ok=True)

# Run the steps
basin_gdf_path = os.path.join(base_folder, 'Basins_Shapefile')
basin_gdf = gpd.read_file(basin_gdf_path)

process_well_headers(wells_headers_folder, basin_gdf, headers_output_folder)

headers_file = os.path.join(headers_output_folder, 'wellheaderswithbasins.csv')
process_production_files(wells_production_folder, headers_file, merged_output_folder)

In [ ]:
split_by_basin(merged_output_folder, split_output_folder)

processed_files = []

# Run the function once
processed_files = concat_by_basin(split_output_folder, final_output_folder, basin_gdf, processed_files)
